### Kaggle Competition
- G-Research Crypto Competition
- EDA 및 전반적인 시계열 데이터 탐색용
- tutorial clone coding
- 개인 스터디를 위한 주석 추가
- 참고 url : https://www.kaggle.com/competitions/g-research-crypto-forecasting
- 사용한 데이터셋은 kaggle competion에서 제공한 데이터 사용 git에 함께 push 하기에는 용량이 커서 올리지 않음

### 라이브러리

In [1]:
import os
import pandas as pd
import plotly.graph_objects as go

### 파라미터

In [2]:
DATA_DIR = os.path.dirname(os.path.dirname(os.path.abspath(os.getcwd())))
TRAIN_DATA_FNM = "crypto_kaggle/train.csv"
ASSET_DATA_FNM = "crypto_kaggle/asset_details.csv"

### 사용자 정의 함수

In [3]:
def show_candle_chart(input_df: pd.DataFrame,
                      referenc_df: pd.DataFrame,
                      asset_id: int) -> None:
    """
    암호화폐 가격 정보 캔들 차트 시각화
    """
    fig = go.Figure(data=[
                    go.Candlestick(x=input_df.index, 
                                   open=input_df['Open'],
                                   high=input_df['High'],
                                   low=input_df['Low'],
                                   close=input_df['Close'])
                    ]
                )
    # 그래프 layout 세팅
    title = referenc_df['Asset_Name'][referenc_df['Asset_ID']==asset_id].values[0]
    fig.update_layout(
        title=dict(
            text='<b>{}</b>'.format(title),
            x=0.5,
            y=0.87,
            font=dict(
                family="Arial",
                size=25,
                color="#000000"
            )
        )
    )
    fig.show()

### EDA

#### 데이터 feature
- timestamp : Second Unix timestamps (분단위 데이터를 나타냄)
- Asset_ID : 암호화폐 종류를 의미함(ex. Asset_ID=1 -> bitcoin을 의미함)
- Count : 해당 interval 동안 거래 횟수
- Open : 해당 interval에서 시가
- High : 해당 interval에서 최고가
- Low : 해당 interval에서 최저가
- Close : 해당 interval에서 종가
- Volume : 거래량
- VWAP : 해당 interval간 거래량의 가중치가 반영된 평균 가격, 거래 데이터의 집계된 형태
- Target : 15분 이상 경과한 자산에 대한 잔여 로그값

In [4]:
# train 데이터 불러오기 및 확인
df = pd.read_csv(os.path.join(DATA_DIR, TRAIN_DATA_FNM))
df.head(10)

,timestamp,Asset_ID,Count,Open,High,Low,Close,Volume,VWAP,Target
0,1514764860,2,40.0,2376.580000,2399.5000,2357.1400,2374.590000,19.233005,2373.116392,-0.004218
1,1514764860,0,5.0,8.530000,8.5300,8.5300,8.530000,78.380000,8.530000,-0.014399
2,1514764860,1,229.0,13835.194000,14013.8000,13666.1100,13850.176000,31.550062,13827.062093,-0.014643
3,1514764860,5,32.0,7.659600,7.6596,7.6567,7.657600,6626.713370,7.657713,-0.013922
4,1514764860,7,5.0,25.920000,25.9200,25.8740,25.877000,121.087310,25.891363,-0.008264
5,1514764860,6,173.0,738.302500,746.0000,732.5100,738.507500,335.987856,738.839291,-0.004809
6,1514764860,9,167.0,225.330000,227.7800,222.9800,225.206667,411.896642,225.197944,-0.009791
7,1514764860,11,7.0,329.090000,329.8800,329.0900,329.460000,6.635710,329.454118,NaN
8,1514764920,2,53.0,2374.553333,2400.9000,2354.2000,2372.286667,24.050259,2371.434498,-0.004079
9,1514764920,0,7.0,8.530000,8.5300,8.5145,8.514500,71.390000,8.520215,-0.015875


In [5]:
# asset detail 데이터 불러오기 및 확인
asset_detail_df = pd.read_csv(os.path.join(DATA_DIR, ASSET_DATA_FNM))
asset_detail_df.head(10)

,Asset_ID,Weight,Asset_Name
0,2,2.397895,Bitcoin Cash
1,0,4.304065,Binance Coin
2,1,6.779922,Bitcoin
3,5,1.386294,EOS.IO
4,7,2.079442,Ethereum Classic
5,6,5.894403,Ethereum
6,9,2.397895,Litecoin
7,11,1.609438,Monero
8,13,1.791759,TRON
9,12,2.079442,Stellar


### Candle 차트 시각화

- 캔들 차트의 길이는 시가와 종가의 차이를 의미한다
- 캔들 차트의 색이 빨강색이면, 종가가 시가보다 낮다는 의미
- 캔들 차트의 색이 초록색이면, 종가가 시가보다 높다는 의미

In [6]:
# Bitcoin(Asset_id == 1) 200개 최근 데이터부터 가져와서 세팅
btc = df[df['Asset_ID']==1].set_index("timestamp")
sample_btc = btc.iloc[-200:, :]

In [21]:
# Bitcoin 캔들 차트 시각화
# x축 : timestamp
fig = go.Figure(data=[
                    go.Candlestick(x=sample_btc.index, 
                                   open=sample_btc['Open'],
                                   high=sample_btc['High'],
                                   low=sample_btc['Low'],
                                   close=sample_btc['Close'])
                    ]
                )
# 그래프 layout 세팅
title = asset_detail_df['Asset_Name'][asset_detail_df['Asset_ID']==1].values[0]
fig.update_layout(
    title=dict(
        text='<b>{}</b>'.format(title),
        x=0.5,
        y=0.87,
        font=dict(
            family="Arial",
            size=25,
            color="#000000"
        )
    )
)
fig.show()

In [7]:
# Ethereum(Asset_id==6) 데이터 정보 확인 및 sample 데이터 200개 추출
ethr=df[df['Asset_ID']==6].set_index("timestamp")
sample_ethr=ethr.iloc[-200:, :]
ethr.info(show_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1956200 entries, 1514764860 to 1632182400
Data columns (total 9 columns):
 #   Column    Non-Null Count    Dtype  
---  ------    --------------    -----  
 0   Asset_ID  1956200 non-null  int64  
 1   Count     1956200 non-null  float64
 2   Open      1956200 non-null  float64
 3   High      1956200 non-null  float64
 4   Low       1956200 non-null  float64
 5   Close     1956200 non-null  float64
 6   Volume    1956200 non-null  float64
 7   VWAP      1956200 non-null  float64
 8   Target    1955860 non-null  float64
dtypes: float64(8), int64(1)
memory usage: 149.2 MB


In [26]:
# Ethereum 캔들차트
show_candle_chart(sample_ethr, asset_detail_df, 6)

### 전처리

In [17]:
# inspect ethereum data null data
ethr.isna().sum()

Asset_ID      0
Count         0
Open          0
High          0
Low           0
Close         0
Volume        0
VWAP          0
Target      340
dtype: int64

In [18]:
# timstamp를 인덱스로 데이터 재정의된 데이터프레임 확인
btc.head()

,Asset_ID,Count,Open,High,Low,Close,Volume,VWAP,Target
timestamp,,,,,,,,,
1514764860,1,229.0,13835.194,14013.8,13666.11,13850.176,31.550062,13827.062093,-0.014643
1514764920,1,235.0,13835.036,14052.3,13680.00,13828.102,31.046432,13840.362591,-0.015037
1514764980,1,528.0,13823.900,14000.4,13601.00,13801.314,55.061820,13806.068014,-0.010309
1514765040,1,435.0,13802.512,13999.0,13576.28,13768.040,38.780529,13783.598101,-0.008999
1514765100,1,742.0,13766.000,13955.9,13554.44,13724.914,108.501637,13735.586842,-0.008079


In [20]:
# timestamp 값을 datetime type으로 변경 후 확인
# 시작과 끝 일부만 확인
beggin_btc = btc.index[0].astype('datetime64[s]')
end_btc = btc.index[-1].astype('datetime64[s]')
beggin_ethr = ethr.index[0].astype('datetime64[s]')
end_ethr = ethr.index[-1].astype('datetime64[s]')

print('BTC data 시작 : {} , 끝 : {}'.format(beggin_btc, end_btc))
print('Ethereum data 시작 : {} , 끝 : {}'.format(beggin_ethr, end_ethr))

BTC data 시작 : 2018-01-01T00:01:00 , 끝 : 2021-09-21T00:00:00
Ethereum data 시작 : 2018-01-01T00:01:00 , 끝 : 2021-09-21T00:00:00


In [10]:
# 각 timestamp 간 간격을 확인하여 비어 있는 값이 있는지 확인 
(ethr.index[1:] - ethr.index[:-1]).value_counts()

60      1956043
120         100
180          23
300           5
540           4
240           3
360           2
480           2
600           2
2220          1
420           1
780           1
900           1
1380          1
2580          1
1320          1
1980          1
1020          1
1860          1
7200          1
960           1
4740          1
3120          1
1080          1
Name: timestamp, dtype: int64